Chapter 04 검증 메트릭
------

### 주요 측정 항목

분류 학습에서 주로 사용하는 메트릭
- Accuracy
- Precision
- recall
- F1-score
- AUC(Area Under the ROC curve)
- 로그 손실
- P@K(정밀도 at k)
- AP@k(평균정밀도 at k)
- MAP@k(Mean Average Precision at k)

회귀학습에서 일반적으로 사용하는 메트릭
- MAE(Mean Absolute Error)
- MSE(Mean Squared Error)
- RMSE(Root Mean Squared Error)
- RMSLE(Root Mean Squared Log Error)
- MPE(Mean Percentage Error)
- MAPE(Mean Absolute Percentage Error)
- R^2

이진 분류 문제
- 기흉 판별 문제
- 학습 데이터 / 검증 데이터
  - 학습 데이터의 각각 세트에는 양성 50 / 음성 50장 - 총 100장 샘플
- 샘플의 개수가 같을 때는 일반적으로 정확도, 정밀도, 리콜, F1 Score를 이용

In [1]:
# 정확도 : 모델이 얼마나 정확하게 분류하는지를 측정

def accuracy(y_true, y_pred):
    """
    정확도를 계산하는 함수
    :param y_true: 타겟 값의 목록
    :param y_pred: 예측 값의 목록
    :return: 정확도
    """
    
    correct_counter = 0
    
    for yt, yp in zip(y_true, y_pred):
        if yt == yp:
            correct_counter += 1
            
    return correct_counter / len(y_true)

In [2]:
from sklearn import metrics

l1 = [0, 1, 1, 1, 0, 0, 0, 1]
l2 = [0, 1, 0, 1, 0, 1, 0, 0]

metrics.accuracy_score(l1, l2)

/opt/conda/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


0.625

데이터를 바꿔보자
- 180개의 음성 / 20개의 양성
- 각각의 세트에는 90개의 음성 / 10개의 양성

타겟 변수가 편향되어 있을 경우
- 하나의 범주가 다른 범주보다 훨씬 많을 때
- 모델의 정확도를 높이는 것은 도움이 되지 않음
- 정밀도를 사용하는 것이 이럴 때는 도움

몇 가지 용어
- TP(True Positive) : 이미지가 기흉이 있다고 예측했고, 실제로 기흉이 있는 경우 -> 모델이 양의 범주라고 맞게 예측한 경우
- TN(True Negative) : 이미지가 정상이라고 예측했고, 실제로 기흉이 없는 이미지인 경우 -> 모델이 음의 범주라고 맞게 예측한 경우
- FP(False Positive) : 이미지가 기흉이 있다고 예측했지만, 실제로 기흉이 없는 이미지인 경우 -> 모델이 양의 범주라고 잘못 예측한 경우
- FN(False Negative) : 이미지가 정상이라고 예측했지만, 실제로 기흉이 있는 이미지인 경우 -> 모델이 음의 범주라고 잘못 예측한 경우

In [4]:
def true_positive(y_true, y_pred):
    """
    TP를 계산하는 함수
    :param y_true: 타겟 값의 목록
    :param y_pred: 예측 값의 목록
    :return: TP의 개수
    """

    tp = 0

    for yt, yp in zip(y_true, y_pred):
        if yt == 1 and yp == 1:
            tp += 1

    return tp


def true_negative(y_true, y_pred):
    """
    TN를 계산하는 함수
    :param y_true: 타겟 값의 목록
    :param y_pred: 예측 값의 목록
    :return: TN의 개수
    """

    tn = 0

    for yt, yp in zip(y_true, y_pred):
        if yt == 0 and yp == 0:
            tn += 1

    return tn


def false_positive(y_true, y_pred):
    """
    FP를 계산하는 함수
    :param y_true: 타겟 값의 목록
    :param y_pred: 예측 값의 목록
    :return: FP의 개수
    """

    fp = 0

    for yt, yp in zip(y_true, y_pred):
        if yt == 1 and yp == 0:
            fp += 1

    return fp


def false_negative(y_true, y_pred):
    """
    FN를 계산하는 함수
    :param y_true: 타겟 값의 목록
    :param y_pred: 예측 값의 목록
    :return: FN의 개수
    """

    fn = 0

    for yt, yp in zip(y_true, y_pred):
        if yt == 0 and yp == 1:
            fn += 1

    return fn

In [6]:
l1 = [0, 1, 1, 1, 0, 0, 0, 1]
l2 = [0, 1, 0, 1, 0, 1, 0, 0]

true_positive(l1, l2), true_negative(l1, l2)

(2, 3)

In [7]:
false_positive(l1, l2), false_negative(l1, l2)

(2, 1)

정확도 = (TP + TN) / (TP + FP + FN + TN)

In [9]:
def accuracy_v2(y_true, y_pred):
    """
    tp/tn/fp/fn 을 사용하여 정확도를 계산하는 함수
    :param y_true: 타겟 값의 목록
    :param y_pred: 예측 값의 목록
    :return: 정확도
    """

    tp = true_positive(y_true, y_pred)
    tn = true_negative(y_true, y_pred)
    fp = false_positive(y_true, y_pred)
    fn = false_negative(y_true, y_pred)
    
    accuracy_score = (tp + tn) / (tp + tn + fp + fn)
    
    return accuracy_score

In [10]:
accuracy_v2(l1, l2)

0.625

**정밀도 = TP / (TP + FP)**

예를 들면....
- 편향된 데ㅣ터로 만든 모델이 90개의 음성 이미지 중 80개를 정확하게 예측하고 10개의 양성 이미지 중 8개를 정확하게 예측했다고 가정
- 이 경우 정확도는 88%
- 100개의 이미지 중, 10개의 음성 이미지와 2개의 양성 이미지를 잘못 예측
  - TP : 8, TN : 80, FP : 10, FN : 2
  - 정밀도를 계산하면 8 / (8 + 10) = 0.444
  - 모델이 양성이라고 예측한 경우, 예측한 44.4% 맞았다고 해석할 수 있다.

In [11]:
def precision(y_true, y_pred):
    """
    정밀도를 계산하는 함수
    :param y_true: 타겟 값의 목록
    :param y_pred: 예측 값의 목록
    :return: 정밀도
    """

    tp = true_positive(y_true, y_pred)
    fp = false_positive(y_true, y_pred)

    precision_result = (tp) / (tp + fp)

    return precision_result

In [12]:
precision(l1, l2)

0.5